In [39]:
import glob
import urllib
from collections import defaultdict
import os
from bs4 import BeautifulSoup

In [5]:
files_to_check = glob.glob('sample_htmls_to_download/*')

In [31]:
fmap = defaultdict(list)
for f in files_to_check:
    f_i = os.path.basename(f)
    f_p = urllib.parse.unquote(f_i)
    d = urllib.parse.urlparse(f_p).netloc.replace('www.', '')
    fmap[d].append(f)

fortworthgov.granicus.com
durhamcounty.granicus.com
denver.granicus.com
jaxcityc.granicus.com
----------------------------------

#index > .index-point

attr = time

---------------------------------------------------------------

seattlechannel.org
---------------------------------

.videoIndex > .seekItem

attr = data-seek

---------------------------------------------------------------

newark.granicus.com
---------------------------------

.indexPoints > a

attr = time

In [106]:
list(fmap.keys())

['fortworthgov.granicus.com',
 'durhamcounty.granicus.com',
 'seattlechannel.org',
 'newark.granicus.com',
 'denver.granicus.com',
 'jaxcityc.granicus.com']

In [102]:
outlet_idx = 5
file_idx = 1
i = list(fmap.items())[outlet_idx][1][file_idx]
# ! open $i

In [103]:
f_i = os.path.basename(i)
urllib.parse.unquote(f_i)

'https://jaxcityc.granicus.com/player/clip/5466?view_id=1&redirect=true.html'

In [105]:
soup = BeautifulSoup(open(i).read())

In [111]:
soup.find(attrs={'id': 'index'}).find_all(attrs={'class': 'index-point'})[0].attrs['time']

'11'

In [ ]:
lookup_dict = []

for d in [
    'fortworthgov.granicus.com',
    'durhamcounty.granicus.com',
    'denver.granicus.com',
    'jaxcityc.granicus.com',
]:
    lookup_dict.append({
        'domain': d,
        'first_level': '#index',
        'second_level': '.index-point',
        'attr': 'time'
    })

lookup_dict.append({
    'domain': 'seattlechannel.org',
    'first_level': '.videoIndex',
    'second_level': '.seekItem',
    'attr': 'data-seek',
})


lookup_dict.append({
    'domain': 'newark.granicus.com',
    'first_level': '.indexPoints',
    'second_level': 'a',
    'attr': 'time',
})

import pandas as pd 
lookup_df = pd.DataFrame(lookup_dict).set_index('domain')

In [116]:
lookup_df

,first_level,second_level,attr
domain,,,
fortworthgov.granicus.com,#index,.index-point,time
durhamcounty.granicus.com,#index,.index-point,time
denver.granicus.com,#index,.index-point,time
jaxcityc.granicus.com,#index,.index-point,time
seattlechannel.org,.videoIndex,.seekItem,data-seek
newark.granicus.com,.indexPoints,a,time


In [160]:
for f in files_to_check:
    f_i = os.path.basename(f)
    f_p = urllib.parse.unquote(f_i)
    d = urllib.parse.urlparse(f_p).netloc.replace('www.', '')
    f_level, s_level, attr = lookup_df.loc[d]
    sel = f'{f_level} > {s_level}'
    
    soup = BeautifulSoup(open(f).read())
    index_points = soup.select(sel)

    schedule = []
    for i in index_points:
        schedule.append({
            'time': i.attrs[attr],
            'title': i.get_text().strip()
        })

    if len(schedule) == 0:
        print(f_p)
        
    output_fname = f.replace('.html', '.schedule.csv')
    pd.DataFrame(schedule).to_csv(output_fname)

https://www.seattlechannel.org/CouncilBriefings?videoid=x100805&amp;Mode2=Video.html
https://jaxcityc.granicus.com/player/clip/1109?view_id=1&redirect=true.html
https://denver.granicus.com/player/clip/14283?view_id=180&redirect=true.html
https://jaxcityc.granicus.com/player/clip/2312?view_id=1&redirect=true.html
https://www.seattlechannel.org/videos?videoid=x108058&amp;Mode2=Video.html
https://seattlechannel.org/mayor-and-council/city-council/2022-2023-public-safety-and-human-services/?videoid=x137547&amp;Mode2=Video.html
https://denver.granicus.com/player/clip/14089?view_id=180&redirect=true.html


In [161]:
! ls -lh sample_htmls_to_download/*schedule.csv

-rw-r--r--@ 1 spangher  staff    97B Jun 24 20:30 sample_htmls_to_download/https%3A%2F%2Fdenver.granicus.com%2Fplayer%2Fclip%2F12936%3Fview_id%3D180%26redirect%3Dtrue.schedule.csv
-rw-r--r--@ 1 spangher  staff     3B Jun 24 20:30 sample_htmls_to_download/https%3A%2F%2Fdenver.granicus.com%2Fplayer%2Fclip%2F14089%3Fview_id%3D180%26redirect%3Dtrue.schedule.csv
-rw-r--r--@ 1 spangher  staff     3B Jun 24 20:30 sample_htmls_to_download/https%3A%2F%2Fdenver.granicus.com%2Fplayer%2Fclip%2F14283%3Fview_id%3D180%26redirect%3Dtrue.schedule.csv
-rw-r--r--@ 1 spangher  staff   4.2K Jun 24 20:30 sample_htmls_to_download/https%3A%2F%2Fdenver.granicus.com%2Fplayer%2Fclip%2F14442%3Fview_id%3D180%26redirect%3Dtrue.schedule.csv
-rw-r--r--@ 1 spangher  staff   399B Jun 24 20:30 sample_htmls_to_download/https%3A%2F%2Fdenver.granicus.com%2Fplayer%2Fclip%2F14894%3Fview_id%3D180%26redirect%3Dtrue.schedule.csv
-rw-r--r--@ 1 spangher  staff   1.0K Jun 24 20:30 sample_htmls_to_download/https%3A%2F%2Fdurhamcount